In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
sys.path.insert(0, '/Users/tareen/Desktop/Research_Projects/2020_mavenn_github/mavenn_local')

import mavenn
import logomaker
import seaborn as sns
import re

%matplotlib inline

In [2]:
# Load example data
data_df = mavenn.load_example_dataset('sortseq_full-wt')

# Separate test from data_df
ix_test = data_df['set']=='test'
test_df = data_df[ix_test].reset_index(drop=True)
print(f'test N: {len(test_df):,}')

# Remove test data from data_df
data_df = data_df[~ix_test].reset_index(drop=True)
print(f'training + validation N: {len(data_df):,}')
data_df.head()

test N: 10,269
training + validation N: 40,249


,set,ct_0,ct_1,ct_2,ct_3,ct_4,ct_5,ct_6,ct_7,ct_8,ct_9,x
0,validation,2,0,0,0,0,0,0,0,0,0,AATTGATGTCCGGTAGCTCACTCATTAGGCAGCCAAGGTTTTAGAC...
1,training,0,0,0,1,0,0,0,0,0,0,AATTCATGTGAGTTATGTCTCTCATTAGGCACCCCAGGCTTGAGAT...
2,training,1,0,0,0,0,0,0,0,0,0,AATTAATGTGAGTTAGCTCACTCATTGGACACCCCAGGCTTTACAC...
3,training,0,1,0,0,0,0,0,0,0,0,ACTTAATAAAAGTCAGCTCACTCATTAATCACCCCACGCTCTACAT...
4,training,0,0,0,0,0,3,0,0,0,0,AATTAATGTGAGTTATCTAACTCATTAGGCACCCCAGGCTTTACAC...


In [3]:
# Comptue sequence length and number of bins
L = len(data_df['x'][0])
y_cols = data_df.columns[1:-1]
Y = len(y_cols)
print(f'L={L}, Y={Y}')

L=75, Y=10


In [4]:
from mavenn.src.layers.gpmap import CustomGPMapLayer


# Tensorflow imports
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.initializers import Constant
from tensorflow.keras.layers import Layer, Dense

class ThermodynamicLayer(CustomGPMapLayer):
    """Represents an thermodynamic model of transcription
       regulation in E. Coli at the lac promoter, which 
       contains binding sites for RNAP and CRP.
    """

    def __init__(self, 
                 L_crp,
                 L_rnap,
                 C,
                 regularizer, 
                 *args, **kwargs):
        """Construct layer instance."""
        self.L_crp=L_crp
        self.L_rnap=L_rnap
        self.C=C
        self.regularizer = tf.keras.regularizers.L2(regularizer)
        super().__init__(*args, **kwargs)

    def build(self, input_shape):
        """Build layer."""
        
        # define bias/chemical potential weight for crp
        self.mu_crp = self.add_weight(name='mu_crp',
                                       shape=(1,),
                                       initializer=Constant(0.),
                                       trainable=True,
                                       regularizer=self.regularizer)

        # define bias/chemical potential weight for rnap
        self.mu_rnap = self.add_weight(name='mu_rnap',
                                       shape=(1,),
                                       initializer=Constant(0.),
                                       trainable=True,
                                       regularizer=self.regularizer)


        # Define theta_crp_lc parameters
        theta_crp_lc_shape = (1, self.L_crp, self.C)

        theta_crp_lc_init = np.random.randn(*theta_crp_lc_shape)/np.sqrt(self.L_crp)
        self.theta_crp_lc = self.add_weight(name='theta_crp_lc',
                                        shape=theta_crp_lc_shape,
                                        initializer=Constant(theta_crp_lc_init),
                                        trainable=True,
                                        regularizer=self.regularizer)
        
        # Define theta_rnap_lc parameters
        theta_rnap_lc_shape = (1, self.L_rnap, self.C)

        theta_rnap_lc_init = np.random.randn(*theta_rnap_lc_shape)/np.sqrt(self.L_rnap)
        self.theta_rnap_lc = self.add_weight(name='theta_rnap_lc',
                                        shape=theta_rnap_lc_shape,
                                        initializer=Constant(theta_rnap_lc_init),
                                        trainable=True,
                                        regularizer=self.regularizer)
        
        # define interaction term. Not sure if this needs regularization
        self.interaction = self.add_weight(name='interaction',
                               shape=(1,),
                               initializer=Constant(0.),
                               trainable=True,
                               regularizer=self.regularizer)
        
        # define tsat term. Not sure if this needs regularization
        self.tsat = self.add_weight(name='tsat',
                               shape=(1,),
                               initializer=Constant(0.),
                               trainable=True,
                               regularizer=self.regularizer)        
        
        # Call superclass build
        super().build(input_shape)

    def call(self, x_lc):
        """Process layer input and return output.
        
        x_lc: (tensor)
            Input tensor that represents one-hot encoded 
            sequence values. 
        """
        
        # extract locations of binding sites from entire lac-promoter sequence.
        x_crp_lc = x_lc[:,4:108]
        x_rnap_lc = x_lc[:,136:300]
        
        # reshape according to crp and rnap lengths. 
        x_crp_lc = tf.reshape(x_crp_lc, [-1, self.L_crp, self.C])
        x_rnap_lc = tf.reshape(x_rnap_lc, [-1, self.L_rnap, self.C])
                
        # compute delta G for crp    
        phi_crp = self.mu_crp + \
              tf.reshape(K.sum(self.theta_crp_lc * x_crp_lc, axis=[1, 2]),
                         shape=[-1, 1])
            
        # compute delta G for rnap
        phi_rnap = self.mu_rnap + \
              tf.reshape(K.sum(self.theta_rnap_lc * x_rnap_lc, axis=[1, 2]),
                         shape=[-1, 1])            
        
        # compute rate of transcription
        t = (self.tsat)*(K.exp(-phi_crp)+K.exp(-phi_rnap)+K.exp(-phi_crp-phi_rnap-self.interaction))/(1+K.exp(-phi_crp)+K.exp(-phi_rnap)+K.exp(-phi_crp-phi_rnap-self.interaction))
        
        # return rate of transcription
        return t


In [ ]:
# define custom gp_map parameters dictionary
gpmap_kwargs = {'L_crp':26,
                'L_rnap':41,
                'C':4,
                'regularizer':0.005}

# Create model
model = mavenn.Model(L=L, 
                     Y=Y,
                     alphabet='dna', 
                     regression_type='MPA', 
                     gpmap_type='custom',
                     gpmap_kwargs=gpmap_kwargs,
                     custom_gpmap=ThermodynamicLayer)

In [ ]:
# Set training data
model.set_data(x=data_df['x'],
               y=data_df[y_cols],
               validation_flags=(data_df['set']=='validation'),
               shuffle=True)

# Fit model to data
model.fit(learning_rate=.0005,
          epochs=2000,
          batch_size=200,
          early_stopping=True,
          early_stopping_patience=25,
          linear_initialization=True)


N = 40,249 observations set as training data.
Using 24.7% for validation.
Data shuffled.
Time to set data: 0.608 sec.
Epoch 1/2000
152/152 [==============================] - 2s 6ms/step - loss: 663.7367 - I_var: -0.0015 - val_loss: 665.3579 - val_I_var: -6.4517e-04
Epoch 2/2000
152/152 [==============================] - 0s 2ms/step - loss: 660.0104 - I_var: -3.8917e-04 - val_loss: 658.8937 - val_I_var: -3.0497e-04
Epoch 3/2000
152/152 [==============================] - 0s 2ms/step - loss: 651.1711 - I_var: -1.6168e-04 - val_loss: 653.1945 - val_I_var: -1.3896e-04
Epoch 4/2000
152/152 [==============================] - 0s 2ms/step - loss: 646.2539 - I_var: 8.6601e-05 - val_loss: 648.0873 - val_I_var: 6.2466e-04
Epoch 5/2000
152/152 [==============================] - 0s 2ms/step - loss: 641.2505 - I_var: 0.0022 - val_loss: 642.5858 - val_I_var: 0.0073
Epoch 6/2000
152/152 [==============================] - 0s 3ms/step - loss: 633.8534 - I_var: 0.0335 - val_loss: 625.8810 - val_I_var: 0.0

Epoch 57/2000
152/152 [==============================] - 0s 3ms/step - loss: 528.7666 - I_var: 0.5248 - val_loss: 538.3342 - val_I_var: 0.5193
Epoch 58/2000
152/152 [==============================] - 0s 2ms/step - loss: 535.7103 - I_var: 0.5246 - val_loss: 538.3080 - val_I_var: 0.5192
Epoch 59/2000
152/152 [==============================] - 0s 2ms/step - loss: 533.5565 - I_var: 0.5248 - val_loss: 537.9628 - val_I_var: 0.5207
Epoch 60/2000
152/152 [==============================] - 0s 3ms/step - loss: 531.0642 - I_var: 0.5246 - val_loss: 537.8702 - val_I_var: 0.5207
Epoch 61/2000
152/152 [==============================] - 0s 3ms/step - loss: 531.3769 - I_var: 0.5254 - val_loss: 537.6061 - val_I_var: 0.5218
Epoch 62/2000
152/152 [==============================] - 0s 2ms/step - loss: 534.9054 - I_var: 0.5262 - val_loss: 537.6574 - val_I_var: 0.5212
Epoch 63/2000
152/152 [==============================] - 0s 2ms/step - loss: 530.5363 - I_var: 0.5267 - val_loss: 537.3468 - val_I_var: 0.5226

152/152 [==============================] - 0s 2ms/step - loss: 529.2665 - I_var: 0.5433 - val_loss: 532.6874 - val_I_var: 0.5364
Epoch 115/2000
152/152 [==============================] - 0s 2ms/step - loss: 528.8700 - I_var: 0.5431 - val_loss: 532.2554 - val_I_var: 0.5386
Epoch 116/2000
152/152 [==============================] - 0s 3ms/step - loss: 527.4465 - I_var: 0.5435 - val_loss: 532.4904 - val_I_var: 0.5372
Epoch 117/2000
152/152 [==============================] - 0s 3ms/step - loss: 529.0925 - I_var: 0.5423 - val_loss: 532.0795 - val_I_var: 0.5392
Epoch 118/2000
152/152 [==============================] - 0s 3ms/step - loss: 525.8535 - I_var: 0.5430 - val_loss: 532.2454 - val_I_var: 0.5379
Epoch 119/2000
152/152 [==============================] - 0s 2ms/step - loss: 530.1305 - I_var: 0.5433 - val_loss: 532.0603 - val_I_var: 0.5389
Epoch 120/2000
152/152 [==============================] - 0s 3ms/step - loss: 524.9605 - I_var: 0.5442 - val_loss: 532.0541 - val_I_var: 0.5388
Epoch 1

In [ ]:
# Show training history
print('On test data:')
x_test = test_df['x'].values
y_test = test_df[y_cols].values

# Compute likelihood information
I_var, dI_var =  model.I_variational(x=x_test, y=y_test)
print(f'I_var_test: {I_var:.3f} +- {dI_var:.3f} bits') 

# Compute predictive information
I_pred, dI_pred = model.I_predictive(x=x_test, y=y_test)
print(f'I_pred_test: {I_pred:.3f} +- {dI_pred:.3f} bits')

I_var_hist = model.history['I_var']
val_I_var_hist = model.history['val_I_var']

fig, ax = plt.subplots(1,1,figsize=[4,4])
ax.plot(I_var_hist, label='I_var_train')
ax.plot(val_I_var_hist, label='I_var_val')
#ax.axhline(I_var, color='C2', linestyle=':', label='I_var_test')
#ax.axhline(I_pred, color='C3', linestyle=':', label='I_pred_test')
ax.legend()
ax.set_xlabel('epochs')
ax.set_ylabel('bits')
ax.set_title('training hisotry')
#ax.set_ylim([0, I_pred*1.2]);

In [ ]:
interaction_term = model.get_nn().layers[2].get_weights()[4]/1.62
crp_weights = model.get_nn().layers[2].get_weights()[2][0]
rnap_weights = model.get_nn().layers[2].get_weights()[3][0]
print(f'interaction term = {interaction_term[0]:.3f} k_cal/mol')

In [ ]:
crp_df = pd.DataFrame(crp_weights,columns=model.alphabet)
rnap_df = pd.DataFrame(rnap_weights,columns=model.alphabet)

In [ ]:
# # Get G-P map parameters in matrix form
# theta = model.get_theta(gauge='uniform')

# Create grid in phi space
phi_lim = [-4, 4]
phi_grid = np.linspace(phi_lim[0], phi_lim[1], 1000)

# Create array of allowable y values
Y = model.model.Y    # Y = number of bins
y_lim = [-.5, Y-.5]
y_all = range(Y)

# Compute matrix of p(y|phi) values
measurement_process = model.p_of_y_given_phi(y_all, phi_grid)

# # Create figure with two panels
fig, axs = plt.subplots(1,3,figsize=[18,4])

logomaker.Logo(-crp_df,ax=axs[0],center_values=True)
logomaker.Logo(-rnap_df,ax=axs[1],center_values=True)

# # Right panel: draw measurement process as heatmap
ax = axs[2]
im = ax.imshow(measurement_process,
               cmap='Greens',
               extent=phi_lim+y_lim,
               vmin=0,
               origin='lower',
               interpolation='nearest',
               aspect="auto")
ax.set_yticks(y_all)
ax.set_ylabel('bin number (y)')
ax.set_xlabel('latent phenotype ($\phi$)')
ax.set_title('measurement process')
cb = plt.colorbar(im)
cb.set_label('probability  $p(y|\phi)$', rotation=-90, va="bottom")

# Fix up plot
fig.tight_layout(w_pad=3)
fig.savefig('thermodynmic_custom_gpmap_mpa_visualization_sort_seq.png',bbox_index='tight',dpi=300)
# plt.show()